In [27]:
import os
import re
import csv
import pickle
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [2]:
def read_table(path):
    print('read', path)
    headers = None
    table = []

    fp = open(path, encoding='utf8', errors='ignore')

    for line in tqdm(fp):
        line = line.strip()
        if line:
            if headers is None:
                headers = line.split(',')
                continue

            row = line.split(',')
            if len(row) == len(headers):
                table.append(row)
#     df = pd.DataFrame(table[:20], columns=headers)
#     emotion = []
#     for x in tqdm(table):
#         text = x[6]
#         m = re.findall(r'\[[\u4e00-\u9fff]{1,3}\]', text)
#         if m:
#             emotion += m
#     sample = [
#         x
#         for x in sorted(list(Counter(emotion).items()), key=lambda x: x[1], reverse=True)
#         if x[1] >= 500
#     ]
#     print(len(sample))
#     print(sample[:10])
    return table

In [3]:
def get_labels():
    labels_dict = {
        'happy': [
            '哈哈',
            '偷笑',
            '嘻嘻',
            '耶',
            '太开心',
            '太開心',
            '(^_^)',
            '(*^__^*)',
            '(^o^)',
            '(^.^)',
            'O(∩_∩)O',
        ],
        'sad': [
            '[泪]',
            '[涙]',
            '[傷心]',
            '[悲傷]',
            '[失望]',
            '[伤心]',
            '(T_T)',
            '(T.T)',
            '(T^T)',
            '(ᅲ.ᅲ)',
            '(╯_╰)',
        ],
        'angry': [
            '[怒]',
            '[抓狂]',
            '[哼]',
            '[左哼哼]',
            '[右哼哼]',
            '[怒骂]',
            '[鄙视]',
            '(╰_╯)',
        ],
        'disgust': [
            '[汗]',
            '[晕]',
            '[暈]',
            '[鄙视]', 
            '[鄙視]', 
            '[黑線]', 
            '[囧]', 
            '[黑线]', 
            '[吐]',
        ],
        'fear': [
            '[可怜]', 
            '[生病]', 
            '[委屈]',
        ],
        'suprise': [
            '[吃惊]', 
            '[吃驚]',
            'OMG',
            '(0.o)',
            '(O_o)',
            '(@_@)',
        ],
        'neutral': [
            '[酷]', 
            '[疑问]', 
            '[握手]', 
            '[疑問]', 
            '[可爱]', 
            '[可愛]', 
            '[微风]', 
            '[浮云]', 
            '[浮雲]', 
            '[兔子]', 
            '[思考]', 
            '[困]',
        ],
    }
    labels = {}
    for k, v in labels_dict.items():
        for vv in v:
            labels[vv] = k
    
    return labels

In [4]:
LABELS = get_labels()

In [5]:
print(LABELS)

{'[囧]': 'disgust', '[汗]': 'disgust', '[生病]': 'fear', '[鄙视]': 'angry', '[可愛]': 'neutral', '[思考]': 'neutral', '[酷]': 'neutral', '[失望]': 'sad', '[黑線]': 'disgust', '(T.T)': 'sad', '太開心': 'happy', '[涙]': 'sad', '[怒]': 'angry', '(O_o)': 'suprise', '[黑线]': 'disgust', '(ᅲ.ᅲ)': 'sad', '[吐]': 'disgust', '[泪]': 'sad', '[兔子]': 'neutral', '[怒骂]': 'angry', '(T_T)': 'sad', '[困]': 'neutral', '[鄙視]': 'disgust', '[可爱]': 'neutral', '太开心': 'happy', '(0.o)': 'suprise', '[伤心]': 'sad', '[微风]': 'neutral', '(╯_╰)': 'sad', '[浮雲]': 'neutral', '偷笑': 'happy', '[疑問]': 'neutral', '[晕]': 'disgust', '[傷心]': 'sad', 'O(∩_∩)O': 'happy', '[右哼哼]': 'angry', '[悲傷]': 'sad', '耶': 'happy', '(*^__^*)': 'happy', '[吃惊]': 'suprise', '[浮云]': 'neutral', 'OMG': 'suprise', '哈哈': 'happy', '(T^T)': 'sad', '(^.^)': 'happy', '[委屈]': 'fear', '[抓狂]': 'angry', '[吃驚]': 'suprise', '[左哼哼]': 'angry', '(@_@)': 'suprise', '[可怜]': 'fear', '[疑问]': 'neutral', '[哼]': 'angry', '(^o^)': 'happy', '嘻嘻': 'happy', '[暈]': 'disgust', '[握手]': 'neutral', '(^_^)'

In [19]:
MIN_LENGTH = 10

def extract_emotion(table):
    emotion_from_table = []
    for x in tqdm(table):
        if len(x) <= 6:
            continue
        text = x[6]
        # 转发取最后一个
        if '//' in text:
            text = text.split('//')[-1]
        # remove at someone
        text = re.sub(r'@[a-zA-Z_0-9\u4e00-\u9fff]+[:：]*(\s+|$)', '', text)
        # remove hash
        text = re.sub(r'#[^#]+#\s*', '', text)
        # without face
        text_without_face = re.sub(r'\s*\[[^\]]+\]\s*', '', text)

        m = re.findall(r'[\u4e00-\u9fff]+', text_without_face)

        if len(m) < MIN_LENGTH:
            continue

        count = 0
        signal_in_text = []
        tag_in_text = None
        for signal, tag in LABELS.items():
            if signal in text:
                if tag == tag_in_text:
                    signal_in_text.append(signal)
                    continue
                elif count >= 1:
                    signal_in_text = None
                    tag_in_text = None
                    break
                else:
                    count += 1
                    signal_in_text.append(signal)
                    tag_in_text = tag

        if tag_in_text is not None:
            clean_text = text_without_face
            for s in signal_in_text:
                clean_text = clean_text.replace(s, '')
            emotion_from_table.append((
                clean_text, text, tag_in_text, signal_in_text))
    return emotion_from_table

In [23]:
root_dir = '/media/qhduan/Seagate Expansion Drive/DATASETS/weibo/'

paths = []
for dirpath, _, filenames in os.walk(root_dir):
    paths += [
        os.path.join(dirpath, x)
        for x in filenames
        if x.endswith('.csv') and 'week' in x
    ]
paths = sorted(paths)

In [24]:
print(len(paths))

52


In [25]:
data = []
for path in paths:
    table = read_table(path)
    emotion_from_table = extract_emotion(table)
    print(len(emotion_from_table))
    data += emotion_from_table

45398it [00:00, 453915.50it/s]

read /media/qhduan/Seagate Expansion Drive/DATASETS/weibo/week1.csv


4790111it [00:16, 296170.25it/s]
100%|██████████| 4633268/4633268 [00:26<00:00, 172901.54it/s]
0it [00:00, ?it/s]

19083
read /media/qhduan/Seagate Expansion Drive/DATASETS/weibo/week10.csv


3882568it [00:13, 286975.40it/s]
100%|██████████| 3752958/3752958 [00:21<00:00, 174759.86it/s]
0it [00:00, ?it/s]

...

19516
read /media/qhduan/Seagate Expansion Drive/DATASETS/weibo/week8.csv


4739982it [00:19, 239152.73it/s]
100%|██████████| 4595408/4595408 [00:26<00:00, 174398.24it/s]
0it [00:00, ?it/s]

18540
read /media/qhduan/Seagate Expansion Drive/DATASETS/weibo/week9.csv


4010777it [00:12, 315467.09it/s]
100%|██████████| 3878452/3878452 [00:23<00:00, 163804.89it/s]

17527


In [26]:
print(len(data))

989154


In [28]:
with open('data.pkl', 'wb') as fp:
    pickle.dump(data, fp)